In [ ]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
from collections import namedtuple
from itertools import chain, product
from projects.tablet.features import Port, Button, CameraBump
from components.fasteners.hex import HexBoreHole

import math

set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

In [ ]:
TABLET_LENGTH = 240
TABLET_WIDTH = 158
TABLET_HEIGHT = 10
TABLET_ROUND_RADIUS = 10
TABLET_CHAMFER = 2
SCREW_BUMP_LENGTH = 12

CASE_THICKNESS = 2

In [ ]:
class TabletShape(BasePartObject):
    def __init__(
        self,
        tablet_length,
        tablet_width,
        tablet_height,
        tablet_body_radius,
        tablet_chamfer,
        rotation = (0,0,0),
        align=(Align.CENTER, Align.CENTER, Align.MIN),
        mode=Mode.ADD
    ):
        with BuildPart() as tablet:
            Box(tablet_length, tablet_width, tablet_height)
            fillet(tablet.edges().filter_by(Axis.Z), tablet_body_radius)
            chamfer(tablet.faces().filter_by(Axis.Z).edges(), tablet_chamfer)
        super().__init__(tablet.part, rotation, align, mode)
        self.side_faces = tablet.part.faces().filter_by(GeomType.PLANE).sort_by(Axis.Z)[5:9]
        self.back_face = tablet.part.faces().sort_by(Axis.Z).first

In [ ]:
with BuildPart() as tablet:
    tablet_shape = TabletShape(TABLET_LENGTH, TABLET_WIDTH, TABLET_HEIGHT, TABLET_ROUND_RADIUS, TABLET_CHAMFER)
    camera_bump = CameraBump(
        tablet_shape.back_face,
        6,
        1,
        (10,10),
        align=(Align.MIN, Align.MIN, Align.MIN)
    )
    usb_c_port = Port(
        tablet_shape.side_faces[3],
        (4, 10, 8), 
        (0,0),
        align=(Align.CENTER, Align.CENTER, Align.MAX)
    )
    power_button = Button(
        tablet_shape.side_faces[0],
        (2, 10, 3),
        (0, 5),
        align=(Align.CENTER, Align.MAX, Align.CENTER)
    )
    volume_button = Button(
        tablet_shape.side_faces[2],
        (2, 20, 3),
        (0, 5),
        align=(Align.CENTER, Align.MIN, Align.CENTER)
    )

In [ ]:
with BuildPart() as tablet_case:
    Box(TABLET_LENGTH + CASE_THICKNESS, TABLET_WIDTH + CASE_THICKNESS, TABLET_HEIGHT, align=(Align.CENTER, Align.CENTER, Align.MIN))
    Box(TABLET_LENGTH + CASE_THICKNESS, TABLET_WIDTH + CASE_THICKNESS, CASE_THICKNESS, align=(Align.CENTER, Align.CENTER, Align.MAX))
    chamfer(tablet_case.edges().filter_by(Axis.Z), TABLET_ROUND_RADIUS / 2)

    add(tablet, mode=Mode.SUBTRACT)
    
    add(camera_bump.cutter(1), mode=Mode.SUBTRACT)
    add(power_button.cutter(4), mode=Mode.SUBTRACT)
    add(volume_button.cutter(4), mode=Mode.SUBTRACT)
    add(usb_c_port.cutter(4), mode=Mode.SUBTRACT)
    
    screw_bump_faces : ShapeList[Face] = ShapeList(map(lambda x: x.offset(SCREW_BUMP_LENGTH), (tablet_case.faces()
        .filter_by(GeomType.PLANE)
        .filter_by(Axis.Z, reverse=True)
        .filter_by_position(Axis.Z, TABLET_CHAMFER, TABLET_HEIGHT - TABLET_CHAMFER)
        .filter_by(Axis.X, reverse=True)
        .filter_by(Axis.Y, reverse=True)
    )))

    for l in screw_bump_faces:
        with BuildSketch(Plane(l).reverse(), mode=Mode.ADD):
            Rectangle(TABLET_HEIGHT + CASE_THICKNESS, math.sqrt(2) * TABLET_ROUND_RADIUS + 5) # chamfer total width + length of minimum feature offset from sides
        extrude(until=Until.NEXT)

    drill_faces = ShapeList([
        x for x in tablet_case.faces().filter_by(
            Plane.XZ.rotated((0,0,45))
        ) + tablet_case.faces().filter_by(
            Plane.XZ.rotated((0,0,-45))
        ) if Plane(x) not in [Plane(y) for y in screw_bump_faces]
    ])
    bolt_faces = drill_faces.sort_by(Axis.X)[:2] + drill_faces.sort_by(Axis.X, reverse=True)[:2]
    nut_faces = drill_faces.sort_by(Axis.X)[2:6]

    bump_ends = tablet_case.faces().sort_by(Axis.X)[2:4] + tablet_case.faces().sort_by(Axis.X, reverse=True)[2:4]
    fillet(bump_ends.edges().filter_by(Axis.Z), TABLET_CHAMFER)
    
    for l in nut_faces:
        with Locations(l):
            HexBoreHole(2.5, 4, 2, rotation=(0,0,90), mode=Mode.SUBTRACT)

In [ ]:
with BuildPart() as bolt:
    Cylinder(4, 5, align=(Align.CENTER, Align.CENTER, Align.MIN))
    Cylinder(2.5, 25, align=(Align.CENTER, Align.CENTER, Align.MAX))

with BuildPart() as nut:
    with BuildSketch():
        RegularPolygon(4, 6, rotation=90)
        Circle(2.5, mode=Mode.SUBTRACT)
    extrude(amount=2)

bolts = ShapeList([Plane(l) * bolt.part for l in bolt_faces])
nuts = ShapeList([Plane(l) * nut.part for l in nut_faces])

In [ ]:
square_length = 40

bb = tablet_case.part.bounding_box()

points = ShapeList([
    (bb.min.X, bb.max.Y),
    (bb.max.X, bb.max.Y),
    (bb.max.X, bb.min.Y),
    (bb.min.X, bb.min.Y),
])
inner_point = (bb.size.X - bb.size.Y) / 2

with BuildLine(mode=Mode.PRIVATE) as rays:
    for i, point in enumerate(points):
        PolarLine(point, (bb.size.X - bb.size.Y), -i * 90 - 45)

with BuildLine() as bottom_splitter_bound:
    bottom_rays = rays.edges().sort_by(Axis.Y)[:2]
    Line(bottom_rays.first @ 0, bottom_rays.last @ 0)
    Line(bottom_rays.first @ 1, bottom_rays.last @ 1)
    add(bottom_rays)

with BuildLine() as top_splitter_bound:
    top_rays = rays.edges().sort_by(Axis.Y, reverse=True)[:2]
    Line(top_rays.first @ 0, top_rays.last @ 0)
    Line(top_rays.first @ 1, top_rays.last @ 1)
    add(top_rays)

with BuildLine() as left_splitter_bound:
    left_rays = rays.edges().sort_by(Axis.X)[:2]
    Line(left_rays.first @ 0, left_rays.last @ 0)
    right_end = Line(
        (-(left_rays.last @ 1).Y, (left_rays.first @ 1).Y),
        (-(left_rays.last @ 1).Y, (left_rays.last @ 1).Y)
    )
    Line(left_rays.first @ 1, right_end @ 0)
    Line(left_rays.last @ 1, right_end @ 1)
    add(left_rays)

with BuildLine() as right_splitter_bound:
    right_rays = rays.edges().sort_by(Axis.X, reverse=True)[:2]
    Line(right_rays.first @ 0, right_rays.last @ 0)
    left_end = Line(
        (-(right_rays.last @ 1).Y, (right_rays.first @ 1).Y),
        (-(right_rays.last @ 1).Y, (right_rays.last @ 1).Y)
    )
    Line(right_rays.first @ 1, left_end @ 0)
    Line(right_rays.last @ 1, left_end @ 1)
    add(right_rays)

with BuildLine() as center_bound:
    add([right_end, left_end, Line(right_end @ 0, left_end @ 1), Line(right_end @ 1, left_end @ 0)])

In [ ]:
with BuildPart() as bottom_piece:
    add(tablet_case)
    with BuildSketch():
        add(bottom_splitter_bound.edges())
        make_face()
    extrude(amount = TABLET_HEIGHT + CASE_THICKNESS, both=True, mode=Mode.INTERSECT)
    bottom_piece.part.name = "bottom"

with BuildPart() as top_piece:
    add(tablet_case)
    with BuildSketch():
        add(top_splitter_bound.edges())
        make_face()
    extrude(amount = TABLET_HEIGHT + CASE_THICKNESS, both=True, mode=Mode.INTERSECT)
    top_piece.part.name = "top"

with BuildPart() as left_piece:
    add(tablet_case)
    with BuildSketch():
        add(left_splitter_bound.edges())
        make_face()
    extrude(amount = TABLET_HEIGHT + CASE_THICKNESS, both=True, mode=Mode.INTERSECT)
    left_piece.part.name = "left"

with BuildPart() as right_piece:
    add(tablet_case)
    with BuildSketch():
        add(right_splitter_bound.edges())
        make_face()
    extrude(amount = TABLET_HEIGHT + CASE_THICKNESS, both=True, mode=Mode.INTERSECT)
    right_piece.part.name = "right"

with BuildPart() as center_piece:
    add(tablet_case)
    with BuildSketch():
        add(center_bound.edges())
        make_face()
    extrude(amount = TABLET_HEIGHT + CASE_THICKNESS, both=True, mode=Mode.INTERSECT)
    center_piece.part.name = "center"

tablet.part.name = "tablet"

tablet_case_pieces = Compound(
    children=[
        Compound(
            ShapeList([bottom_piece.part, top_piece.part, left_piece.part, right_piece.part, center_piece.part]),
            label="tablet case",
            color="gold"
        ),
        Compound(
            tablet.part,
            color="gray",
            label="tablet"
        ),
        Compound(
            bolts,
            color="silver",
            label="bolts"
        ),
        Compound(
            nuts,
            color="silver",
            label="nuts"
        )
    ],
    label="Tablet Case"
)

show(
    tablet_case_pieces,    
)